<a href="https://colab.research.google.com/github/simecek/dspracticum2024/blob/main/lesson02/FashionMNIST_SimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [324]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [325]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**2. Data Preparation**

In [326]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

**3. Neural Network Model**

In [327]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=24*24*32, out_features=64)
        self.drop = nn.Dropout(0.15)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.drop = nn.Dropout(0.15)
        self.fc3 = nn.Linear(in_features=32, out_features=16)
        self.drop = nn.Dropout(0.15)
        self.fc4 = nn.Linear(in_features=16, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.fc4(x)
        return x

model = SimpleCNN().to(device)
model

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=18432, out_features=64, bias=True)
  (drop): Dropout(p=0.15, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [328]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.25)

**5. Training loop**

In [329]:
for epoch in range(100):  # Train for 5 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/100], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/100], Loss: 1.1272
Epoch [2/100], Loss: 0.5688
Epoch [3/100], Loss: 0.4709
Epoch [4/100], Loss: 0.4183
Epoch [5/100], Loss: 0.3788
Epoch [6/100], Loss: 0.3497
Epoch [7/100], Loss: 0.3241
Epoch [8/100], Loss: 0.3029
Epoch [9/100], Loss: 0.2894
Epoch [10/100], Loss: 0.2682
Epoch [11/100], Loss: 0.2547
Epoch [12/100], Loss: 0.2444
Epoch [13/100], Loss: 0.2321
Epoch [14/100], Loss: 0.2192
Epoch [15/100], Loss: 0.2063
Epoch [16/100], Loss: 0.1981
Epoch [17/100], Loss: 0.1882
Epoch [18/100], Loss: 0.1756
Epoch [19/100], Loss: 0.1692
Epoch [20/100], Loss: 0.1617
Epoch [21/100], Loss: 0.1505
Epoch [22/100], Loss: 0.1456
Epoch [23/100], Loss: 0.1409
Epoch [24/100], Loss: 0.1299
Epoch [25/100], Loss: 0.1261
Epoch [26/100], Loss: 0.1230
Epoch [27/100], Loss: 0.1146
Epoch [28/100], Loss: 0.1073
Epoch [29/100], Loss: 0.1053
Epoch [30/100], Loss: 0.1020
Epoch [31/100], Loss: 0.0974
Epoch [32/100], Loss: 0.0904
Epoch [33/100], Loss: 0.0890
Epoch [34/100], Loss: 0.0852
Epoch [35/100], Loss: 0

**6. Evaluation on the test set**

In [330]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 90.49%
